In [0]:
%pip install dbl-discoverx
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("catalogs", "*", "Catalogs")
dbutils.widgets.text("schemas", "*", "Schemas")
dbutils.widgets.text("tables", "*", "Tables")
dbutils.widgets.text("owner","metastore_admin","owner")

In [0]:
catalogs = dbutils.widgets.get("catalogs")
schemas = dbutils.widgets.get("schemas")
tables = dbutils.widgets.get("tables")
owner = dbutils.widgets.get("owner")
from_table_statement = ".".join([catalogs, schemas, tables])

In [0]:
from discoverx import DX

dx = DX()

In [0]:
def update_owner(table_info):
  catalog_owner_alter_sql = f""" ALTER CATALOG `{table_info.catalog}` SET OWNER TO `{owner}`"""
  schema_owner_alter_sql = f""" ALTER SCHEMA `{table_info.catalog}`.`{table_info.schema}` SET OWNER TO `{owner}`"""
  table_owner_alter_sql = f""" ALTER TABLE `{table_info.catalog}`.`{table_info.schema}`.`{table_info.table}` SET OWNER TO `{owner}`"""
  try:
    print(f"Executing {catalog_owner_alter_sql}")
    spark.sql(catalog_owner_alter_sql)

    print(f"Executing {schema_owner_alter_sql}")
    spark.sql(schema_owner_alter_sql)

    print(f"Executing {table_owner_alter_sql}")
    spark.sql(table_owner_alter_sql)
  except Exception as exception: 
    print(f"  Exception occurred while updating owner: {exception}")

dx.from_tables(from_table_statement).map(update_owner)

In [0]:
from concurrent.futures import ThreadPoolExecutor

def update_owner_conn(conn,owner):
    print(f"Executing {conn}")
    try:
        spark.sql(f"ALTER CONNECTION {conn} OWNER TO {owner}")
    except Exception as exception: 
        print(f"Exception occurred while updating {conn} TO {owner}: {exception}")

conns = spark.sql("""
            select collect_set(connection_name) from system.information_schema.connections where connection_owner != 'metastore_admin'
          """).first()[0]

with ThreadPoolExecutor(max_workers=100) as executor:
        executor.map(update_owner_conn,conns,owner)

In [0]:
from concurrent.futures import ThreadPoolExecutor

def update_owner_cred(cred,owner):
    print(f"Executing {cred}")
    try:
        spark.sql(f"ALTER STORAGE CREDENTIAL {cred} OWNER TO {owner}")
    except Exception as exception: 
        print(f"Exception occurred while updating {cred} TO {owner}: {exception}")

creds = spark.sql("""
            select collect_set(storage_credential_name) from system.information_schema.storage_credentials where storage_credential_owner != 'metastore_admin'
          """).first()[0]

with ThreadPoolExecutor(max_workers=100) as executor:
        executor.map(update_owner_cred,creds,owner)

In [0]:
from concurrent.futures import ThreadPoolExecutor

def update_owner_cred(cred,owner):
    print(f"Executing {cred}")
    try:
        spark.sql(f"ALTER SERVICE CREDENTIAL {cred} OWNER TO {owner}")
    except Exception as exception: 
        print(f"Exception occurred while updating {cred} TO {owner}: {exception}")

creds = spark.sql("""
            select collect_set(credential_name) from system.information_schema.credentials where credential_owner != 'metastore_admin'
          """).first()[0]

with ThreadPoolExecutor(max_workers=100) as executor:
        executor.map(update_owner_cred,creds,owner)

In [0]:
from concurrent.futures import ThreadPoolExecutor

def update_owner_extl(extl):
    print(f"Executing {extl}")
    try:
        spark.sql(f"ALTER EXTERNAL LOCATION `{extl}` OWNER TO `{owner}`")
    except Exception as exception: 
        print(f"Exception occurred while updating {extl} TO {owner}: {exception}")

extls = spark.sql("""
            select collect_set(external_location_name) from system.information_schema.external_locations where external_location_owner != '{owner}'
          """).first()[0]

with ThreadPoolExecutor(max_workers=100) as executor:
        executor.map(update_owner_extl,extls)

In [0]:
from concurrent.futures import ThreadPoolExecutor

def update_owner_extl(vol):
    print(f"Executing {vol}")
    try:
        spark.sql(f"ALTER VOLUME `{vol}` SET OWNER TO `{owner}`")
    except Exception as exception: 
        print(f"Exception occurred while updating {vol} TO {owner}: {exception}")

extls = spark.sql("""
            select collect_set(volume_name) from system.information_schema.volumes where volume_owner != '{owner}'
          """).first()[0]

with ThreadPoolExecutor(max_workers=100) as executor:
        executor.map(update_owner_extl,extls)